In [ ]:
# Imports
import pandas as pd
import ast
import re
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
file_path = '/content/drive/MyDrive/LLM_thesis/filtered_df.parquet'
merged_df = pd.read_parquet(file_path)

#### Starting with gpt-2

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2Model
from sklearn.manifold import TSNE
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = GPT2Model.from_pretrained("openai-community/gpt2")

In [ ]:
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# Getting average-pooled sentence embedding from GPT-2
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        last_hidden_state = outputs.last_hidden_state
        embedding = last_hidden_state.mean(dim=1).squeeze()
    return embedding.cpu().numpy()

In [ ]:
# Extracting embeddings for all sentences
embeddings = []
for sentence in tqdm(filtered_df["sentence"], desc="Extracting GPT-2 embeddings"):
    emb = get_embedding(sentence)
    embeddings.append(emb)

In [ ]:
embeddings_df = pd.DataFrame(embeddings)  # shape = (n_sentences, 768)
embeddings_df["stereotype"] = filtered_df["stereotype"].values # added 1 col
embeddings_df.shape